Experiment 1: Vary the number of hidden inputs 
where n = 20,50,100

In [1]:
#Importing the necessary libraries
import math 
import numpy as np 
from scipy.special import expit
from sklearn.metrics import confusion_matrix, accuracy_score
import random 
import csv
import matplotlib.pyplot as plt

In [3]:
# Loading the training data
file_train = ("mnist_train.csv","r")
train_data = csv.reader(file_train)
train = np.array(list(train_data))
#train

In [4]:
# Loading the test data
file_test = ("mnist_test.csv","r")
test_data = csv.reader(file_test)
test = np.array(list(test_data))
#test

In [5]:
np.random.shuffle(train)
train = train[0:60000]

In [6]:
#Initializing the values of bias,learning rate and momentum
bias = 1
learning_rate = 0.1
m = 0.9

In [7]:
#To vary the hidden inputs, change the n value to 50 and 100 
#Number of hidden inputs


In [9]:
#Appling the weights

#weights from inner layer to hidden layer
weight_ih = np.random.uniform(-0.05,0.05,(785,n))
#print(weight_ih.shape)

#weight from hidden layer to outer layer
weight_ho = np.random.uniform(-0.05,0.05,(n+1,10))
#print(weight_ho.shape)

In [10]:
#Store the delta weight from hidden layer to output layer
previous_weight_ho = np.zeros((n+1,10))

#Store the delta weight from the input to the hidden layer
previous_weight_ih = np.zeros((785,n))

In [11]:
#A matrix to store the activation h1...hk
h1_input = np.zeros((1,n+1))
h1_input[0,0] = 1

In [12]:
#Define Perceptron
def multi_perceptron(epoch,input_ds,set_flag):
    global weight_ih, weight_ho, previous_weight_ih, previous_weight_ho
    predict_list =[]
    actual_list = []
    for i in range(input_ds.shape[0]):
        target_class = input_ds[i,0].astype('int')
        actual_list.append(target_class)
        xi = input_ds[i].astype('float16')/255
        xi[0] = bias
        xi = xi.reshape(1,785)
        
        z_h1 = np.dot(xi,weight_ih)
        sigmoid_h1 = expit(z_h1)
        print("sigmoid_h1",sigmoid_h1.shape)
        
        h1_input[0,1:] = sigmoid_h1
        print("h1 input",h1_input)
        print(h1_input.shape)
        z_o1 = np.dot(h1_input,weight_ho)
        sigmoid_o1 = expit(z_o1)
        print("sigmoid_o1",sigmoid_o1)
        
        predict = np.argmax(sigmoid_o1)
        print(predict)
        predict_list.append(predict)
        print(type(sigmoid_o1))
        print(sigmoid_o1.shape)
        
        if epoch>0 and set_flag == 1:
            #print error for output unit
            tk = np.zeros((1,10))+0.1
            tk[0,target_class] = 0.9
            print(tk)
            error_o1 = sigmoid_o1 * (1 - sigmoid_o1) * (tk - sigmoid_o1) 
            print("error_o1 shape for ",epoch," ", error_o1.shape)
            
            #print error for hidden unit
            error_h1 = sigmoid_h1 * (1 - sigmoid_h1) * np.dot(error_o1,weight_ho[1:,:].T)
            print("error_h1 shape for ",epoch," ",error_h1.shape)
            
            #Update the weights of hidden layer to outer layer
            
            delta_weight_ho = (learning_rate * error_o1 * h1_input.T) + (m * previous_weight_ho)
            previous_weight_ho = delta_weight_ho
            print(delta_weight_ho.shape)
            weight_ho = weight_ho + delta_weight_ho
            
            #Updating the weights of input to output
            
            delta_weight_ih = (learning_rate * error_h1 * x.T) + (m * previous_weight_ih)
            previous_weight_ih = delta_weight_ih
            print(delta_weight_ih.shape)
            weight_ih = weight_ih + delta_weight_ih
            
        accuracy = (np.array(predict_list) == np.array(actual_list)).sum()/float(len(actual_list)) * 100
        print("Length of actual list after ",epoch," : ",len(actual_list))
        print("Length of prediction  list after ",epoch," : ",len(predict_list))
        
        if (set_flag == 0):
            print("Confusion Matrix for epoch",epoch)
            print(confusion_matrix(actual_list,predict_list))
            
        return accuracy
    



In [13]:
def store_accuracy(accuracy_index,accuracy,input_ds):
    with open(input_ds, 'a', newline = '') as myfile:
        wr = csv.writer(myfile)
        wr.writerow([accuracy_index,accuracy])

In [ ]:
for each in range(50):
    train_accuarcy = multi_perceptron(each,train,1)
    test_accuracy = multi_perceptron(each,test,0)
    store_accuracy(each,train_accuracy,'train_no='+str(m)+'.csv')
    store_accuracy(each,test_accuracy,'test_no='+str(m)+'.csv')          

In [ ]:
# Plotting the graph
x1,y1 = np.loadtxt("train_no.csv",delimiter=',',unpack=True)
x2,y2 = np.loadtxt("test_no.csv",delimiter=',',unpack=True)
plt.plot(x1,y1,label="Training Set")
plt.plot(x2,y2,label="Test Set")
plt.xlabel("Epochs")
plt.ylabel('Accuracy (%)')
plt.title ('Experiment 1 For n=20')
plt.legend()
plt.show()